# I Create the table to train

In [78]:
import pandas as pd
from utils import choice_form

In [64]:
df_train = pd.read_csv("../data/intermediate/df_to_train.csv")
df_conclusion  = pd.read_csv("../data/intermediate/df_conclusion.csv")

In [65]:
df_train = df_train.merge(df_conclusion, on = "id_seq" )

In [44]:
df_train.head()

,id_seq,sentenced,choice_str,task_form,human_response,has_conclusion,index_conclusion
0,1_0_R,Some models are managers and All models are clerks,"['All managers are clerks', 'All clerks are managers', 'Some managers are clerks', 'Some clerks are managers', 'Some managers are not clerks', 'Some clerks are not managers', 'No managers are clerks', 'No clerks are managers', 'NVC']",IA4,"('Iac', True)",True,2
1,1_1_R,No divers are carpenters and All linguists are carpenters,"['All divers are linguists', 'All linguists are divers', 'Some divers are linguists', 'Some linguists are divers', 'Some divers are not linguists', 'Some linguists are not divers', 'No divers are linguists', 'No linguists are divers', 'NVC']",EA3,"('Eac', True)",True,4
2,1_2_R,All therapists are climbers and Some skaters are therapists,"['All climbers are skaters', 'All skaters are climbers', 'Some climbers are skaters', 'Some skaters are climbers', 'Some climbers are not skaters', 'Some skaters are not climbers', 'No climbers are skaters', 'No skaters are climbers', 'NVC']",AI2,"('Ica', True)",True,2
3,1_3_R,All bankers are golfers and All golfers are teachers,"['All bankers are teachers', 'All teachers are bankers', 'Some bankers are teachers', 'Some teachers are bankers', 'Some bankers are not teachers', 'Some teachers are not bankers', 'No bankers are teachers', 'No teachers are bankers', 'NVC']",AA1,"('Aac', True)",True,0
4,1_4_R,Some boxers are not opticians and All boxers are actuaries,"['All opticians are actuaries', 'All actuaries are opticians', 'Some opticians are actuaries', 'Some actuaries are opticians', 'Some opticians are not actuaries', 'Some actuaries are not opticians', 'No opticians are actuaries', 'No actuaries are opticians', 'NVC']",OA4,"('Oca', True)",True,5


In [66]:
df = df_train[df_train["has_conclusion"]==True]

In [6]:
df = df[["id_seq","sentenced","choice_str"]]

In [7]:
df.to_csv("../data/intermediate/df_valid_to_train_fs.csv")

# II Analyse des résultats

In [67]:
df_trained = pd.read_csv("../data/intermediate/few_shot_trained_full.csv")
df_conclusion  = pd.read_csv("../data/intermediate/df_conclusion.csv")

In [68]:
df_trained = df_trained.merge(df_conclusion, on = "id_seq" )

In [55]:
df_trained.head()

,Unnamed: 0,id_seq,sentenced,few_shot_pred,task_form,human_response,has_conclusion,index_conclusion,choice_str
0,0,1_0_R,Some models are managers and All models are clerks,Some managers are clerks,IA4,"('Iac', True)",True,2,"['All managers are clerks', 'All clerks are managers', 'Some managers are clerks', 'Some clerks are managers', 'Some managers are not clerks', 'Some clerks are not managers', 'No managers are clerks', 'No clerks are managers', 'NVC']"
1,1,1_1_R,No divers are carpenters and All linguists are carpenters,Some linguists are divers,EA3,"('Eac', True)",True,4,"['All divers are linguists', 'All linguists are divers', 'Some divers are linguists', 'Some linguists are divers', 'Some divers are not linguists', 'Some linguists are not divers', 'No divers are linguists', 'No linguists are divers', 'NVC']"
2,2,1_2_R,All therapists are climbers and Some skaters are therapists,Some skaters are climbers,AI2,"('Ica', True)",True,2,"['All climbers are skaters', 'All skaters are climbers', 'Some climbers are skaters', 'Some skaters are climbers', 'Some climbers are not skaters', 'Some skaters are not climbers', 'No climbers are skaters', 'No skaters are climbers', 'NVC']"
3,3,1_3_R,All bankers are golfers and All golfers are teachers,All bankers are teachers,AA1,"('Aac', True)",True,0,"['All bankers are teachers', 'All teachers are bankers', 'Some bankers are teachers', 'Some teachers are bankers', 'Some bankers are not teachers', 'Some teachers are not bankers', 'No bankers are teachers', 'No teachers are bankers', 'NVC']"
4,6,1_6_R,All gamblers are sculptors and Some bakers are not sculptors,Some gamblers are sculptors,AO3,"('Oca', True)",True,5,"['All gamblers are bakers', 'All bakers are gamblers', 'Some gamblers are bakers', 'Some bakers are gamblers', 'Some gamblers are not bakers', 'Some bakers are not gamblers', 'No gamblers are bakers', 'No bakers are gamblers', 'NVC']"


In [69]:
import ast

def select_best(prediction,choices):
    """ determine the answer of the model as the conclusion with the 
    highest probability
    """
    # We clean the prediction
    prediction = prediction[1:]
    if isinstance(choices,str):
        choices = ast.literal_eval(choices)
    
    for conclusion in choices:
        if prediction == conclusion:
            return choices.index(prediction)
        
    return - 1  


In [79]:
# We determine what is the form of the few shot answer
df_trained['index_choice'] = df_trained[["few_shot_pred","choice_str"]].apply(lambda x: select_best(x[0],x[1]), axis=1)
df_trained['form_choice'] = df_trained.index_choice.apply(lambda x: choice_form[x])

In [71]:
import ast
df_trained['few_shot_succes'] = df_trained[["index_choice","index_conclusion"]].apply(lambda x: 1 if x[0] in ast.literal_eval(x[1]) else 0, axis=1)

In [97]:
df_result_fewshot = df_trained[['task_form', 'few_shot_succes']].groupby(['task_form']).mean()
df_result_fewshot['task_form'] = df_result_fewshot.index
df_result_fewshot.to_csv("../data/results/df_fewshot_results.csv" ,index=False)

In [91]:
df = df_trained[["task_form", "form_choice"]]
#df_groupby = df.groupby('task_form').form_choice.value_counts()
df_groupby = df.groupby(['task_form',"form_choice"]).size().reset_index(name='counts')


In [92]:
df_groupby

,task_form,form_choice,counts
0,AA1,Aac,74
1,AA1,Iac,2
2,AA1,NVC,28
3,AA2,Aac,4
4,AA2,Aca,23
...,...,...,...
106,IE4,Eac,1
107,IE4,Eca,3
108,IE4,Iac,3
109,IE4,Ica,2


In [93]:
df_groupby.to_csv("../data/results/df_fewshot_group_by.csv" ,index=False)